In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

from time import time
from icecream import ic

from msfm.utils import analysis, parameters
from msfm import fiducial_pipeline

### constants

In [3]:
conf = analysis.load_config()

# tfr_pattern = "/cluster/scratch/athomsen/CosmoGrid/v1/fiducial/DESy3_fiducial_???.tfrecord"
# tfr_pattern = "/Users/arne/data/DESY3/tfrecords/v2/DESy3_fiducial_000.tfrecord"
tfr_pattern = "/pscratch/sd/a/athomsen/DESY3/v2/fiducial/DESy3_fiducial_???.tfrecord"

params = ["Om", "s8", "Aia"]
pert_labels = parameters.get_fiducial_perturbation_labels(params)
n_perts = len(pert_labels)

n_z_bins = 4
global_batch_size = 16
examples_shuffle_buffer = 10
n_readers = 1
n_prefetch = 3
n_batches = 7

23-03-01 05:34:53  analysis.py INF   Loaded the config 


### strategy

In [4]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(['/gpu:0', '/gpu:1'])
# strategy = tf.distribute.get_strategy()

print(tf.config.list_physical_devices("GPU"))
print(strategy.num_replicas_in_sync)

2023-03-01 05:34:54.830821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 05:34:56.547844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78993 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2023-03-01 05:34:56.549043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78993 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-03-01 05:34:56.550049: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
4


# single noise

In [5]:
def dataset_fn(input_context):
    dset = fiducial_pipeline.get_fiducial_dset(
        tfr_pattern,
        pert_labels,
        global_batch_size,
        conf=None,
        n_batches=n_batches,
        i_noise=0,
        n_readers=n_readers,
        n_prefetch=n_prefetch,
        examples_shuffle_buffer=examples_shuffle_buffer,
        input_context=input_context
    )

    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

23-03-01 05:34:57 fiducial_pip INF   Starting to generate the fiducial training set for i_noise = 0 
23-03-01 05:34:57  analysis.py INF   Loaded the config 
23-03-01 05:34:57  analysis.py INF   Loaded the pixel file 
23-03-01 05:34:57  analysis.py INF   Loaded the config 
23-03-01 05:34:57  analysis.py INF   Loaded the pixel file 
0
23-03-01 05:34:58 fiducial_pip INF   Sharding the dataset over 4 replicas 
23-03-01 05:34:58 fiducial_pip INF   Using a local batch size of 4 per replica 
23-03-01 05:34:58 tfrecords.py WAR   Tracing parse_inverse_fiducial 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set th

In [6]:
i = 0
for data_vectors, index in dist_dset:
    print(i)
    i += 1
    
    print(index)

0
(PerReplica:{
  0: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([587, 453, 373, 513])>,
  1: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([224, 221, 571, 710])>,
  2: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([147,  95, 185,  78])>,
  3: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([306, 516, 179, 283])>
}, PerReplica:{
  0: <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>,
  1: <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>,
  2: <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>,
  3: <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>
})
1
(PerReplica:{
  0: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([440, 361, 611,  45])>,
  1: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([312, 183, 538, 370])>,
  2: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 46, 604, 152, 326])>,
  3: <tf.Tensor: shape=(4,), dtype=int64, numpy=array([556,  62, 672, 705

# multi noise

In [7]:
def dataset_fn(input_context):
    dset = fiducial_pipeline.get_fiducial_multi_noise_dset(
        tfr_pattern,
        pert_labels,
        batch_size,
        conf=None,
        n_batches=n_batches,
        n_noise=3,
        n_readers=n_readers,
        n_prefetch=n_prefetch,
        examples_shuffle_buffer=examples_shuffle_buffer,
        input_context=input_context
    )

    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

for data_vectors, index in dist_dset:
    print(data_vectors.shape)

NameError: name 'batch_size' is not defined